# First Neurons

In [ ]:
import numpy as np
import tensorflow as tf

<b>Set Random Seeds for same results</b>

In [ ]:
np.random.seed(101)
tf.set_random_seed(101)

#### The graph we will be building

<br/>
<div>
  <img src="./graph_for_Jose_06-29.jpg"
       alt="The TensorFlow Graph which we will be building for section 6-29."
       width="500px">
</div>
<br/>

<b>Data Setup</b>

Setting Up some Random Data for Demonstration Purposes

In [ ]:
rand_a = np.random.uniform(0, 100, (5, 5))
rand_a

In [ ]:
rand_b = np.random.uniform(0, 100, (5, 1))
rand_b

### Placeholders

In [ ]:
a = tf.placeholder(tf.float32)

In [ ]:
b = tf.placeholder(tf.float32) # we're not using the shape parameter for now.

### Operations

In [ ]:
add_op = a + b  # Could do tf.add(a, b), and look at the docs

In [ ]:
mult_op = a * b # Could do tf.multiply(a, b)

### Running Sessions to create Graphs with Feed Dictionaries

In [ ]:
with tf.Session() as sess:
    
    add_result = sess.run(add_op, feed_dict={a:10, b:20})
    print(add_result)
    
##endof:  with tf.Session() as sess:

In [ ]:
with tf.Session() as sess:
    
    add_result = sess.run(add_op, feed_dict={a:rand_a, b:rand_b})
    print(add_result)
    
    print('\n')
    
    mult_result = sess.run(mult_op, feed_dict={a:rand_a, b:rand_b})
    print(mult_result)
    
##endof:  with tf.Session as sess

<hr/>
<hr/>

## Example Neural Network

<b>Operation Activation Function</b>

<b>Variable Initializer!</b>

We still need to finish off this process with optimization! Let's learn how to do this next.

<hr/>

## Full Network Example

Let's work on a regression example, we are trying to solve a very simple equation:

y = mx + b

y will be the y_labels and x is the x_data. We are trying to figure out the slope and the intercept for the line that best fits our data!

### Artificial Data (Some Made Up Regression Data)

<b>Variables</b>

### Cost Function

### Optimizer

### Initialize Variables

### Create Session and Run!

### Evaluate Results

_That's all for now!_